In [15]:
from Crypto.Cipher import AES, DES3
from Crypto.Random import get_random_bytes
from Crypto.Util.Padding import pad, unpad
from Crypto.Util import Counter
from base64 import b64decode, b64encode

# Triple DES

We will be using the key generation method of choosing 3 distict keys as it is the strongest algorithm against brute force attacks. Each key is 8-bytes, so with 3 keys the total is 24 bytes. Accounting for 24 parity bits, the key length would be 168 bits. But due to "meet-in-the-middle" attack, the effective key length is 112 bits (https://en.wikipedia.org/wiki/Meet-in-the-middle_attack) (https://en.wikipedia.org/wiki/Triple_DES) (https://pycryptodome.readthedocs.io/en/latest/src/cipher/des3.html)

## Key generation

We are generating 3 distinct keys by checking if each part of the key is not equal

In [16]:

def generate_key():
    while True:
        key = get_random_bytes(24)
        try:
            key = DES3.adjust_key_parity(key)
            break
        except ValueError:
            pass

    k1, k2, k3 = key[0: 8], key[8:16], key[16:]

    if k1 != k2 and k2 != k3 and k1 != k3:
        return key, (k1, k2, k3)

key, (k1, k2, k3) = generate_key()
print(key.hex())
print(k1.hex())
print(k2.hex())
print(k3.hex())

0d58028ab5e001800d43268c23251a975b4c315d64c14f3d
0d58028ab5e00180
0d43268c23251a97
5b4c315d64c14f3d


In [ ]:
cipher = DES3.new(key, DES3.MO)
